# 데이터 불러오기

In [ ]:
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploades file "{name}" with length {length}bytes', format(len(uploaded[fn])))

Saving card_Y.csv to card_Y.csv
Saving default of credit card clients_del.csv to default of credit card clients_del.csv
Saving default of credit card clients_dev&del.csv to default of credit card clients_dev&del.csv
Saving default of credit card clients_dev.csv to default of credit card clients_dev.csv
Saving default of credit card clients2.csv to default of credit card clients2.csv
User uploades file "{name}" with length {length}bytes 90031
User uploades file "{name}" with length {length}bytes 1571141
User uploades file "{name}" with length {length}bytes 2235742
User uploades file "{name}" with length {length}bytes 3292663
User uploades file "{name}" with length {length}bytes 2609233


In [ ]:
import pandas as pd

secom_data = pd.read_csv('default of credit card clients2.csv', sep = ',')
data_or = secom_data.copy()
secom_labels = pd.read_csv('card_Y.csv')
label_or = secom_labels.copy()

data_dev = pd.read_csv('default of credit card clients_dev.csv', sep = ',')
data_del = pd.read_csv('default of credit card clients_del.csv', sep = ',')
data_dev_del = pd.read_csv('default of credit card clients_dev&del.csv', sep = ',')

# 데이터 정규화

In [ ]:
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()
# 오리지널 데이터
std_scaler.fit(data_or)
std = std_scaler.transform(data_or)
data_or_sk = pd.DataFrame(std, columns = data_or.columns, index = list(data_or.index.values))
# dev 데이터
std_scaler.fit(data_dev)
std = std_scaler.transform(data_dev)
data_dev_sk = pd.DataFrame(std, columns = data_dev.columns, index = list(data_dev.index.values))
# del 데이터
std_scaler.fit(data_del)
std = std_scaler.transform(data_del)
data_del_sk = pd.DataFrame(std, columns = data_del.columns, index = list(data_del.index.values))
# dev+del 데이터
std_scaler.fit(data_dev_del)
std = std_scaler.transform(data_dev_del)
data_dev_del_sk = pd.DataFrame(std, columns = data_dev_del.columns, index = list(data_dev_del.index.values))

# 데이터 훈련셋과 검정셋 나누기 (스케일링 한거)

In [ ]:
from sklearn.model_selection import train_test_split

X_train1, X_test1, Y_train1, Y_test1 = train_test_split(data_or_sk, label_or, test_size = 0.2, random_state = 0)
X_train2, X_test2, Y_train2, Y_test2 = train_test_split(data_dev_sk, label_or, test_size = 0.2, random_state = 0)
X_train3, X_test3, Y_train3, Y_test3 = train_test_split(data_del_sk, label_or, test_size = 0.2, random_state = 0)
X_train4, X_test4, Y_train4, Y_test4 = train_test_split(data_dev_del_sk, label_or, test_size = 0.2, random_state = 0)


In [ ]:
from sklearn.model_selection import train_test_split

X_train11, X_test11, Y_train11, Y_test11 = train_test_split(data_or, label_or, test_size = 0.2, random_state = 0)
X_train21, X_test21, Y_train21, Y_test21 = train_test_split(data_dev, label_or, test_size = 0.2, random_state = 0)
X_train31, X_test31, Y_train31, Y_test31 = train_test_split(data_del, label_or, test_size = 0.2, random_state = 0)
X_train41, X_test41, Y_train41, Y_test41 = train_test_split(data_dev_del, label_or, test_size = 0.2, random_state = 0)

# 데이터 오버샘플링과 아닌거로 나누기

In [ ]:
from imblearn.over_sampling import SMOTE

# scale 한거

# smote = 0, original data
smote1 = SMOTE(random_state = 42)
data_over1, label_over1 = smote1.fit_resample(X_train1, Y_train1)
label_over1.value_counts()

# smote = 0, dev data
smote2 = SMOTE(random_state = 42)
data_over2, label_over2 = smote2.fit_resample(X_train2, Y_train2)
label_over2.value_counts()

# smote = 0, del data
smote3 = SMOTE(random_state = 42)
data_over3, label_over3 = smote3.fit_resample(X_train3, Y_train3)
label_over3.value_counts()

# smote = 0, dev&del data
smote4 = SMOTE(random_state = 42)
data_over4, label_over4 = smote4.fit_resample(X_train4, Y_train4)
label_over4.value_counts()

# smote 안한거, original data
# smote 안한거, dev data
# smote 안한거, del data
# smote 안한거, dev&del data

default payment next month
0                             18661
1                             18661
dtype: int64

# Random Forest

## 그리드 서치

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import numpy as np

from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')

# define the parameters grid
param_grid = {'max_depth': np.arange(3, 10),
             'criterion' : ['gini','entropy'],
             'max_leaf_nodes': [5,10,20,100],
             'min_samples_split': [2, 5, 10, 20]}

# create the grid
grid_tree = GridSearchCV(RandomForestClassifier(), param_grid, cv = 5, scoring= 'f1')
# the cv option will be clear in a few cells

#training
grid_tree.fit(X_train1, Y_train1)
#let's see the best estimator
print(grid_tree.best_estimator_)
#with its score
print(np.abs(grid_tree.best_score_))

RandomForestClassifier(criterion='entropy', max_depth=8, max_leaf_nodes=100) 0.4674145668459264

In [ ]:
# 스케일 한거

from sklearn.ensemble import RandomForestClassifier

RFC_ov1 = RandomForestClassifier(bootstrap=True, criterion='entropy', max_depth=8, max_leaf_nodes=100)
RFC_ov1.fit(data_over1, label_over1)
RFC_pred_ov1 = RFC_ov1.predict(X_test1)

RFC_ov2 = RandomForestClassifier(bootstrap=True, criterion='entropy', max_depth=8, max_leaf_nodes=100)
RFC_ov2.fit(data_over2, label_over2)
RFC_pred_ov2 = RFC_ov2.predict(X_test2)

RFC_ov3 = RandomForestClassifier(bootstrap=True, criterion='entropy', max_depth=8, max_leaf_nodes=100)
RFC_ov3.fit(data_over3, label_over3)
RFC_pred_ov3 = RFC_ov3.predict(X_test3)

RFC_ov4 = RandomForestClassifier(bootstrap=True, criterion='entropy', max_depth=8, max_leaf_nodes=100)
RFC_ov4.fit(data_over4, label_over4)
RFC_pred_ov4 = RFC_ov4.predict(X_test4)

RFC1 = RandomForestClassifier(bootstrap=True, criterion='entropy', max_depth=8, max_leaf_nodes=100)
RFC1.fit(X_train1, Y_train1)
RFC_pred1 = RFC1.predict(X_test1)

RFC2 = RandomForestClassifier(bootstrap=True, criterion='entropy', max_depth=8, max_leaf_nodes=100)
RFC2.fit(X_train2, Y_train2)
RFC_pred2 = RFC2.predict(X_test2)

RFC3 = RandomForestClassifier(bootstrap=True, criterion='entropy', max_depth=8, max_leaf_nodes=100)
RFC3.fit(X_train3, Y_train3)
RFC_pred3 = RFC3.predict(X_test3)

RFC4 = RandomForestClassifier(bootstrap=True, criterion='entropy', max_depth=8, max_leaf_nodes=100)
RFC4.fit(X_train4, Y_train4)
RFC_pred4 = RFC4.predict(X_test4)

#스케일링한거

In [ ]:
# 오버샘플링 O, 오리지널 데이터
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, f1_score
from sklearn.model_selection import cross_val_score, KFold

kf = KFold(n_splits=10)
print(classification_report(Y_test1, RFC_pred_ov1))
print("교차 검증 점수: {:.4f}".format(cross_val_score(RFC_ov1, X_test1, Y_test1, cv=kf).mean()))
print("AUC 점수: {:.4f}".format(roc_auc_score(Y_test1, RFC_pred_ov1)))
print("F1-Score: {:.4f}".format(f1_score(RFC_pred_ov1, Y_test1)))

              precision    recall  f1-score   support

           0       0.88      0.82      0.85      4703
           1       0.48      0.59      0.53      1297

    accuracy                           0.77      6000
   macro avg       0.68      0.70      0.69      6000
weighted avg       0.79      0.77      0.78      6000

교차 검증 점수: 0.8287
AUC 점수: 0.7043
F1-Score: 0.5258


In [ ]:
# 오버샘플링 O, 편차 변수 추가 데이터
print(classification_report(Y_test2, RFC_pred_ov2))
print("교차 검증 점수: {:.4f}".format(cross_val_score(RFC_ov2, X_test2, Y_test2, cv=kf).mean()))
print("AUC 점수: {:.4f}".format(roc_auc_score(Y_test2, RFC_pred_ov2)))
print("F1-Score: {:.4f}".format(f1_score(RFC_pred_ov2, Y_test2)))

              precision    recall  f1-score   support

           0       0.88      0.85      0.86      4703
           1       0.51      0.57      0.54      1297

    accuracy                           0.79      6000
   macro avg       0.69      0.71      0.70      6000
weighted avg       0.80      0.79      0.79      6000

교차 검증 점수: 0.8280
AUC 점수: 0.7100
F1-Score: 0.5383


In [ ]:
# 오버샘플링 O, 변수 삭제 데이터
print(classification_report(Y_test3, RFC_pred_ov3))
print("교차 검증 점수: {:.4f}".format(cross_val_score(RFC_ov3, X_test3, Y_test3, cv=kf).mean()))
print("AUC 점수: {:.4f}".format(roc_auc_score(Y_test3, RFC_pred_ov3)))
print("F1-Score: {:.4f}".format(f1_score(RFC_pred_ov3, Y_test3)))

              precision    recall  f1-score   support

           0       0.88      0.81      0.84      4703
           1       0.46      0.59      0.52      1297

    accuracy                           0.76      6000
   macro avg       0.67      0.70      0.68      6000
weighted avg       0.79      0.76      0.77      6000

교차 검증 점수: 0.8300
AUC 점수: 0.7000
F1-Score: 0.5182


In [ ]:
# 오버샘플링 O, 편차 변수 + 변수 삭제 데이터
print(classification_report(Y_test4, RFC_pred_ov4))
print("교차 검증 점수: {:.4f}".format(cross_val_score(RFC_ov4, X_test4, Y_test4, cv=kf).mean()))
print("AUC 점수: {:.4f}".format(roc_auc_score(Y_test4, RFC_pred_ov4)))
print("F1-Score: {:.4f}".format(f1_score(RFC_pred_ov4, Y_test4)))

              precision    recall  f1-score   support

           0       0.88      0.84      0.86      4703
           1       0.49      0.57      0.53      1297

    accuracy                           0.78      6000
   macro avg       0.68      0.70      0.69      6000
weighted avg       0.79      0.78      0.78      6000

교차 검증 점수: 0.8312
AUC 점수: 0.7027
F1-Score: 0.5264


In [ ]:
# 오버샘플링 X, 오리지널 데이터
print(classification_report(Y_test1, RFC_pred1))
print("교차 검증 점수: {:.4f}".format(cross_val_score(RFC1, X_test1, Y_test1, cv=kf).mean()))
print("AUC 점수: {:.4f}".format(roc_auc_score(Y_test1, RFC_pred1)))
print("F1-Score: {:.4f}".format(f1_score(RFC_pred1, Y_test1)))

              precision    recall  f1-score   support

           0       0.84      0.96      0.90      4703
           1       0.70      0.36      0.47      1297

    accuracy                           0.83      6000
   macro avg       0.77      0.66      0.68      6000
weighted avg       0.81      0.83      0.81      6000

교차 검증 점수: 0.8265
AUC 점수: 0.6574
F1-Score: 0.4727


In [ ]:
# 오버샘플링 X, 편차 변수 추가 데이터
print(classification_report(Y_test2, RFC_pred2))
print("교차 검증 점수: {:.4f}".format(cross_val_score(RFC2, X_test2, Y_test2, cv=kf).mean()))
print("AUC 점수: {:.4f}".format(roc_auc_score(Y_test2, RFC_pred2)))
print("F1-Score: {:.4f}".format(f1_score(RFC_pred2, Y_test2)))

              precision    recall  f1-score   support

           0       0.84      0.96      0.90      4703
           1       0.70      0.36      0.48      1297

    accuracy                           0.83      6000
   macro avg       0.77      0.66      0.69      6000
weighted avg       0.81      0.83      0.81      6000

교차 검증 점수: 0.8280
AUC 점수: 0.6589
F1-Score: 0.4759


In [ ]:
# 오버샘플링 X, 변수 삭제 데이터
print(classification_report(Y_test3, RFC_pred3))
print("교차 검증 점수: {:.4f}".format(cross_val_score(RFC3, X_test3, Y_test3, cv=kf).mean()))
print("AUC 점수: {:.4f}".format(roc_auc_score(Y_test3, RFC_pred3)))
print("F1-Score: {:.4f}".format(f1_score(RFC_pred3, Y_test3)))

              precision    recall  f1-score   support

           0       0.84      0.95      0.90      4703
           1       0.69      0.36      0.48      1297

    accuracy                           0.83      6000
   macro avg       0.77      0.66      0.69      6000
weighted avg       0.81      0.83      0.81      6000

교차 검증 점수: 0.8295
AUC 점수: 0.6596
F1-Score: 0.4768


In [ ]:
# 오버샘플링 X, 편차 변수 + 변수 삭제 데이터
print(classification_report(Y_test4, RFC_pred4))
print("교차 검증 점수: {:.4f}".format(cross_val_score(RFC4, X_test4, Y_test4, cv=kf).mean()))
print("AUC 점수: {:.4f}".format(roc_auc_score(Y_test4, RFC_pred4)))
print("F1-Score: {:.4f}".format(f1_score(RFC_pred4, Y_test4)))

              precision    recall  f1-score   support

           0       0.85      0.96      0.90      4703
           1       0.72      0.37      0.49      1297

    accuracy                           0.83      6000
   macro avg       0.78      0.66      0.69      6000
weighted avg       0.82      0.83      0.81      6000

교차 검증 점수: 0.8293
AUC 점수: 0.6635
F1-Score: 0.4855


# 스케일링 안한거

In [ ]:
from imblearn.over_sampling import SMOTE

#scale 안한거

# smote = 0, original data
smote11 = SMOTE(random_state = 42)
data_over11, label_over11 = smote11.fit_resample(X_train11, Y_train11)
label_over11.value_counts()

# smote = 0, dev data
smote21 = SMOTE(random_state = 42)
data_over21, label_over21 = smote21.fit_resample(X_train21, Y_train21)
label_over21.value_counts()

# smote = 0, del data
smote31 = SMOTE(random_state = 42)
data_over31, label_over31 = smote31.fit_resample(X_train31, Y_train31)
label_over31.value_counts()

# smote = 0, dev&del data
smote41 = SMOTE(random_state = 42)
data_over41, label_over41 = smote41.fit_resample(X_train41, Y_train41)
label_over41.value_counts()

default payment next month
0                             18661
1                             18661
dtype: int64

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RFC_ov11 = RandomForestClassifier(bootstrap=True, criterion='entropy', max_depth=8, max_leaf_nodes=100)
RFC_ov11.fit(data_over11, label_over11)
RFC_pred_ov11 = RFC_ov11.predict(X_test11)

RFC_ov21 = RandomForestClassifier(bootstrap=True, criterion='entropy', max_depth=8, max_leaf_nodes=100)
RFC_ov21.fit(data_over21, label_over21)
RFC_pred_ov21 = RFC_ov21.predict(X_test21)

RFC_ov31 = RandomForestClassifier(bootstrap=True, criterion='entropy', max_depth=8, max_leaf_nodes=100)
RFC_ov31.fit(data_over31, label_over31)
RFC_pred_ov31 = RFC_ov31.predict(X_test31)

RFC_ov41 = RandomForestClassifier(bootstrap=True, criterion='entropy', max_depth=8, max_leaf_nodes=100)
RFC_ov41.fit(data_over41, label_over41)
RFC_pred_ov41 = RFC_ov41.predict(X_test41)

RFC11 = RandomForestClassifier(bootstrap=True, criterion='entropy', max_depth=8, max_leaf_nodes=100)
RFC11.fit(X_train11, Y_train11)
RFC_pred11 = RFC11.predict(X_test11)

RFC21 = RandomForestClassifier(bootstrap=True, criterion='entropy', max_depth=8, max_leaf_nodes=100)
RFC21.fit(X_train21, Y_train21)
RFC_pred21 = RFC21.predict(X_test21)

RFC31 = RandomForestClassifier(bootstrap=True, criterion='entropy', max_depth=8, max_leaf_nodes=100)
RFC31.fit(X_train31, Y_train31)
RFC_pred31 = RFC31.predict(X_test31)

RFC41 = RandomForestClassifier(bootstrap=True, criterion='entropy', max_depth=8, max_leaf_nodes=100)
RFC41.fit(X_train41, Y_train41)
RFC_pred41 = RFC41.predict(X_test41)

In [ ]:
# 오버샘플링 O, 오리지널 데이터
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, f1_score
from sklearn.model_selection import cross_val_score, KFold

kf = KFold(n_splits=10)
print(classification_report(Y_test11, RFC_pred_ov11))
print("교차 검증 점수: {:.4f}".format(cross_val_score(RFC_ov11, X_test11, Y_test11, cv=kf).mean()))
print("AUC 점수: {:.4f}".format(roc_auc_score(Y_test11, RFC_pred_ov11)))
print("F1-Score: {:.4f}".format(f1_score(RFC_pred_ov11, Y_test11)))

              precision    recall  f1-score   support

           0       0.87      0.81      0.84      4703
           1       0.45      0.56      0.50      1297

    accuracy                           0.76      6000
   macro avg       0.66      0.69      0.67      6000
weighted avg       0.78      0.76      0.77      6000

교차 검증 점수: 0.8268
AUC 점수: 0.6882
F1-Score: 0.5026


In [ ]:
# 오버샘플링 O, 편차 변수 추가 데이터
print(classification_report(Y_test21, RFC_pred_ov21))
print("교차 검증 점수: {:.4f}".format(cross_val_score(RFC_ov21, X_test21, Y_test21, cv=kf).mean()))
print("AUC 점수: {:.4f}".format(roc_auc_score(Y_test21, RFC_pred_ov21)))
print("F1-Score: {:.4f}".format(f1_score(RFC_pred_ov21, Y_test21)))

              precision    recall  f1-score   support

           0       0.87      0.82      0.84      4703
           1       0.46      0.55      0.50      1297

    accuracy                           0.76      6000
   macro avg       0.66      0.69      0.67      6000
weighted avg       0.78      0.76      0.77      6000

교차 검증 점수: 0.8280
AUC 점수: 0.6874
F1-Score: 0.5028


In [ ]:
# 오버샘플링 O, 변수 삭제 데이터
print(classification_report(Y_test31, RFC_pred_ov31))
print("교차 검증 점수: {:.4f}".format(cross_val_score(RFC_ov31, X_test31, Y_test31, cv=kf).mean()))
print("AUC 점수: {:.4f}".format(roc_auc_score(Y_test31, RFC_pred_ov31)))
print("F1-Score: {:.4f}".format(f1_score(RFC_pred_ov31, Y_test31)))

              precision    recall  f1-score   support

           0       0.87      0.81      0.84      4703
           1       0.45      0.55      0.49      1297

    accuracy                           0.76      6000
   macro avg       0.66      0.68      0.67      6000
weighted avg       0.78      0.76      0.76      6000

교차 검증 점수: 0.8287
AUC 점수: 0.6821
F1-Score: 0.4945


In [ ]:
# 오버샘플링 O, 편차 변수 + 변수 삭제 데이터
print(classification_report(Y_test41, RFC_pred_ov41))
print("교차 검증 점수: {:.4f}".format(cross_val_score(RFC_ov41, X_test41, Y_test41, cv=kf).mean()))
print("AUC 점수: {:.4f}".format(roc_auc_score(Y_test41, RFC_pred_ov41)))
print("F1-Score: {:.4f}".format(f1_score(RFC_pred_ov41, Y_test41)))

              precision    recall  f1-score   support

           0       0.87      0.81      0.84      4703
           1       0.45      0.55      0.49      1297

    accuracy                           0.75      6000
   macro avg       0.66      0.68      0.67      6000
weighted avg       0.78      0.75      0.76      6000

교차 검증 점수: 0.8308
AUC 점수: 0.6819
F1-Score: 0.4938


In [ ]:
# 오버샘플링 X, 오리지널 데이터
print(classification_report(Y_test11, RFC_pred11))
print("교차 검증 점수: {:.4f}".format(cross_val_score(RFC11, X_test11, Y_test11, cv=kf).mean()))
print("AUC 점수: {:.4f}".format(roc_auc_score(Y_test11, RFC_pred11)))
print("F1-Score: {:.4f}".format(f1_score(RFC_pred11, Y_test11)))

              precision    recall  f1-score   support

           0       0.84      0.96      0.90      4703
           1       0.69      0.36      0.47      1297

    accuracy                           0.83      6000
   macro avg       0.77      0.66      0.69      6000
weighted avg       0.81      0.83      0.81      6000

교차 검증 점수: 0.8272
AUC 점수: 0.6579
F1-Score: 0.4736


In [ ]:
# 오버샘플링 X, 편차 변수 추가 데이터
print(classification_report(Y_test21, RFC_pred21))
print("교차 검증 점수: {:.4f}".format(cross_val_score(RFC21, X_test21, Y_test21, cv=kf).mean()))
print("AUC 점수: {:.4f}".format(roc_auc_score(Y_test21, RFC_pred21)))
print("F1-Score: {:.4f}".format(f1_score(RFC_pred21, Y_test21)))

              precision    recall  f1-score   support

           0       0.84      0.96      0.90      4703
           1       0.70      0.36      0.47      1297

    accuracy                           0.83      6000
   macro avg       0.77      0.66      0.69      6000
weighted avg       0.81      0.83      0.81      6000

교차 검증 점수: 0.8293
AUC 점수: 0.6584
F1-Score: 0.4748


In [ ]:

# 오버샘플링 X, 변수 삭제 데이터
print(classification_report(Y_test31, RFC_pred31))
print("교차 검증 점수: {:.4f}".format(cross_val_score(RFC31, X_test31, Y_test31, cv=kf).mean()))
print("AUC 점수: {:.4f}".format(roc_auc_score(Y_test31, RFC_pred31)))
print("F1-Score: {:.4f}".format(f1_score(RFC_pred31, Y_test31)))

              precision    recall  f1-score   support

           0       0.85      0.95      0.90      4703
           1       0.69      0.37      0.49      1297

    accuracy                           0.83      6000
   macro avg       0.77      0.66      0.69      6000
weighted avg       0.81      0.83      0.81      6000

교차 검증 점수: 0.8302
AUC 점수: 0.6645
F1-Score: 0.4865


In [ ]:
# 오버샘플링 X, 편차 변수 + 변수 삭제 데이터
print(classification_report(Y_test41, RFC_pred41))
print("교차 검증 점수: {:.4f}".format(cross_val_score(RFC41, X_test41, Y_test41, cv=kf).mean()))
print("AUC 점수: {:.4f}".format(roc_auc_score(Y_test41, RFC_pred41)))
print("F1-Score: {:.4f}".format(f1_score(RFC_pred41, Y_test41)))

              precision    recall  f1-score   support

           0       0.85      0.96      0.90      4703
           1       0.70      0.37      0.48      1297

    accuracy                           0.83      6000
   macro avg       0.77      0.66      0.69      6000
weighted avg       0.82      0.83      0.81      6000

교차 검증 점수: 0.8308
AUC 점수: 0.6625
F1-Score: 0.4830


# XGBOOSTING

## 그리드 서치

In [ ]:
from xgboost import plot_importance
from xgboost import XGBClassifier

xgb = XGBClassifier()

xgb_param_grid = {
    'n_estimators' : [100, 200, 400, 600],
    'learning_rate' : [0.01, 0.05, 0.1, 0.15, 0.2],
    'max_depth' : [4, 6, 8, 10, 12],
}

xgb_grid = GridSearchCV(xgb, param_grid = xgb_param_grid, scoring="f1", n_jobs= -1, verbose = 1)
xgb_grid.fit(data_over1, label_over1)

print(xgb_grid.best_estimator_)
print(np.abs(xgb_grid.best_score_))

XGBClassifier(max_depth=12, n_estimators=600)
0.8470115199979323 -> 정확도 기준
Fitting 5 folds for each of 45 candidates, totalling 225 fits
XGBClassifier(learning_rate=0.2, max_depth=12, n_estimators=400)
0.8309240019524131 -> F1 기준

In [ ]:

from xgboost import XGBClassifier

xgb_ov1 = XGBClassifier(n_estimators=300, learning_rate = 0.01, max_depth = 6)
xgb_ov1.fit(data_over1, label_over1)
xgb_pred_ov1 = xgb_ov1.predict(X_test1)

xgb_ov2 = XGBClassifier(n_estimators=300, learning_rate = 0.01, max_depth = 6)
xgb_ov2.fit(data_over2, label_over2)
xgb_pred_ov2 = xgb_ov2.predict(X_test2)

xgb_ov3 = XGBClassifier(n_estimators=300, learning_rate = 0.01, max_depth = 6)
xgb_ov3.fit(data_over3, label_over3)
xgb_pred_ov3 = xgb_ov3.predict(X_test3)

xgb_ov4 = XGBClassifier(n_estimators=300, learning_rate = 0.01, max_depth = 6)
xgb_ov4.fit(data_over4, label_over4)
xgb_pred_ov4 = xgb_ov4.predict(X_test4)

xgb1 = XGBClassifier(n_estimators=300, learning_rate = 0.01, max_depth = 6)
xgb1.fit(X_train1, Y_train1)
xgb_pred1 = xgb1.predict(X_test1)

xgb2 = XGBClassifier(n_estimators=300, learning_rate = 0.01, max_depth = 6)
xgb2.fit(X_train2, Y_train2)
xgb_pred2 = xgb2.predict(X_test2)

xgb3 = XGBClassifier(n_estimators=300, learning_rate = 0.01, max_depth = 6)
xgb3.fit(X_train3, Y_train3)
xgb_pred3 = xgb3.predict(X_test3)

xgb4 = XGBClassifier(n_estimators=300, learning_rate = 0.01, max_depth = 6)
xgb4.fit(X_train4, Y_train4)
xgb_pred4 = xgb4.predict(X_test4)

In [ ]:
# 오버샘플링 O, 오리지널 데이터
print(classification_report(Y_test1, xgb_pred_ov1))
print("교차 검증 점수: {:.4f}".format(cross_val_score(xgb_ov1, X_test1, Y_test1, cv=kf).mean()))
print("AUC 점수: {:.4f}".format(roc_auc_score(Y_test1, xgb_pred_ov1)))
print("F1-Score: {:.4f}".format(f1_score(xgb_pred_ov1, Y_test1)))

              precision    recall  f1-score   support

           0       0.87      0.85      0.86      4703
           1       0.51      0.55      0.53      1297

    accuracy                           0.79      6000
   macro avg       0.69      0.70      0.70      6000
weighted avg       0.79      0.79      0.79      6000

교차 검증 점수: 0.8273
AUC 점수: 0.7009
F1-Score: 0.5275


In [ ]:
# 오버샘플링 O, 편차 변수 추가 데이터
print(classification_report(Y_test2, xgb_pred_ov2))
print("교차 검증 점수: {:.4f}".format(cross_val_score(xgb_ov2, X_test2, Y_test2, cv=kf).mean()))
print("AUC 점수: {:.4f}".format(roc_auc_score(Y_test2, xgb_pred_ov2)))
print("F1-Score: {:.4f}".format(f1_score(xgb_pred_ov2, Y_test2)))

              precision    recall  f1-score   support

           0       0.87      0.88      0.87      4703
           1       0.54      0.52      0.53      1297

    accuracy                           0.80      6000
   macro avg       0.71      0.70      0.70      6000
weighted avg       0.80      0.80      0.80      6000

교차 검증 점수: 0.8245
AUC 점수: 0.7013
F1-Score: 0.5335


In [ ]:
# 오버샘플링 O, 변수 삭제 데이터
print(classification_report(Y_test3, xgb_pred_ov3))
print("교차 검증 점수: {:.4f}".format(cross_val_score(xgb_ov3, X_test3, Y_test3, cv=kf).mean()))
print("AUC 점수: {:.4f}".format(roc_auc_score(Y_test3, xgb_pred_ov3)))
print("F1-Score: {:.4f}".format(f1_score(xgb_pred_ov3, Y_test3)))

              precision    recall  f1-score   support

           0       0.87      0.85      0.86      4703
           1       0.49      0.54      0.51      1297

    accuracy                           0.78      6000
   macro avg       0.68      0.69      0.69      6000
weighted avg       0.79      0.78      0.78      6000

교차 검증 점수: 0.8287
AUC 점수: 0.6926
F1-Score: 0.5140


In [ ]:
# 오버샘플링 O, 편차 변수 + 변수 삭제 데이터
print(classification_report(Y_test4, xgb_pred_ov4))
print("교차 검증 점수: {:.4f}".format(cross_val_score(xgb_ov4, X_test4, Y_test4, cv=kf).mean()))
print("AUC 점수: {:.4f}".format(roc_auc_score(Y_test4, xgb_pred_ov4)))
print("F1-Score: {:.4f}".format(f1_score(xgb_pred_ov4, Y_test4)))

              precision    recall  f1-score   support

           0       0.87      0.87      0.87      4703
           1       0.52      0.52      0.52      1297

    accuracy                           0.79      6000
   macro avg       0.70      0.70      0.70      6000
weighted avg       0.79      0.79      0.79      6000

교차 검증 점수: 0.8265
AUC 점수: 0.6965
F1-Score: 0.5243


In [ ]:

# 오버샘플링 X, 오리지널 데이터
print(classification_report(Y_test1, xgb_pred1))
print("교차 검증 점수: {:.4f}".format(cross_val_score(xgb1, X_test1, Y_test1, cv=kf).mean()))
print("AUC 점수: {:.4f}".format(roc_auc_score(Y_test1, xgb_pred1)))
print("F1-Score: {:.4f}".format(f1_score(xgb_pred1, Y_test1)))

              precision    recall  f1-score   support

           0       0.84      0.96      0.90      4703
           1       0.69      0.36      0.48      1297

    accuracy                           0.83      6000
   macro avg       0.77      0.66      0.69      6000
weighted avg       0.81      0.83      0.81      6000

교차 검증 점수: 0.8273
AUC 점수: 0.6599
F1-Score: 0.4775


In [ ]:
# 오버샘플링 X, 편차 변수 추가 데이터
print(classification_report(Y_test2, xgb_pred2))
print("교차 검증 점수: {:.4f}".format(cross_val_score(xgb2, X_test2, Y_test2, cv=kf).mean()))
print("AUC 점수: {:.4f}".format(roc_auc_score(Y_test2, xgb_pred2)))
print("F1-Score: {:.4f}".format(f1_score(xgb_pred2, Y_test2)))

              precision    recall  f1-score   support

           0       0.85      0.95      0.90      4703
           1       0.69      0.37      0.48      1297

    accuracy                           0.83      6000
   macro avg       0.77      0.66      0.69      6000
weighted avg       0.81      0.83      0.81      6000

교차 검증 점수: 0.8245
AUC 점수: 0.6610
F1-Score: 0.4796


In [ ]:
# 오버샘플링 X, 변수 삭제 데이터
print(classification_report(Y_test3, xgb_pred3))
print("교차 검증 점수: {:.4f}".format(cross_val_score(xgb3, X_test3, Y_test3, cv=kf).mean()))
print("AUC 점수: {:.4f}".format(roc_auc_score(Y_test3, xgb_pred3)))
print("F1-Score: {:.4f}".format(f1_score(xgb_pred3, Y_test3)))

              precision    recall  f1-score   support

           0       0.84      0.96      0.90      4703
           1       0.70      0.36      0.47      1297

    accuracy                           0.83      6000
   macro avg       0.77      0.66      0.68      6000
weighted avg       0.81      0.83      0.80      6000

교차 검증 점수: 0.8287
AUC 점수: 0.6565
F1-Score: 0.4709


In [ ]:
# 오버샘플링 X, 편차 변수 + 변수 삭제 데이터
print(classification_report(Y_test4, xgb_pred4))
print("교차 검증 점수: {:.4f}".format(cross_val_score(xgb4, X_test4, Y_test4, cv=kf).mean()))
print("AUC 점수: {:.4f}".format(roc_auc_score(Y_test4, xgb_pred4)))
print("F1-Score: {:.4f}".format(f1_score(xgb_pred4, Y_test4)))

              precision    recall  f1-score   support

           0       0.84      0.96      0.90      4703
           1       0.70      0.36      0.48      1297

    accuracy                           0.83      6000
   macro avg       0.77      0.66      0.69      6000
weighted avg       0.81      0.83      0.81      6000

교차 검증 점수: 0.8265
AUC 점수: 0.6592
F1-Score: 0.4764
